# Analyzing Used Car Listings on eBay Kleinanzeigen

The dataset being used for this analysis is downaloaded from eBay Kleinanzeigen, which is a used cars classified section on eBay Germany

Objective of this analysis is to check if brand, prices and mileage have any correlation.

In [1]:
import pandas as pd
import numpy as np

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

The dataset has 20 columns and some of the columns has NaN values and the column names are not in consistent format. Let us clean up the column names first by changing the names from camelcase to snakecase and also use meaningful words.

In [4]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']

# Initial Data Exploration

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-11 22:38:16,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


It looks like nr_of_pictures column has all 0 values and it may not be useful for our analysis. Let's drop the column from dataset of our interest. 

In [6]:
autos['nr_of_pictures'].value_counts()

0    50000
Name: nr_of_pictures, dtype: int64

In [7]:
autos = autos.drop(['nr_of_pictures'], axis =1)

In [8]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'postal_code', 'last_seen'],
      dtype='object')

Same is the case with seller and offer_type fields. They show just one value for the entire dataset. Let's drop these columns.

In [9]:
autos = autos.drop(['seller','offer_type'], axis =1)

price and odometer columns are numeric by nature but they are being stored as text becuase of dollar symobols and denom separators. Lets remove $ and "," and converet them to integer type. 

In [10]:
autos['price'] = autos['price'].str.replace('$','').str.replace(',','').astype(int)

In [11]:
autos['price'] = autos['price'].astype(int)

In [12]:
autos['odometer'] = autos['odometer'].str.replace(',','').str.replace('km','').astype(float)

In [13]:
autos.rename(columns={'odometer':'odometer_km'}, inplace=True)

In [14]:
autos['odometer_km'] = autos['odometer_km'].astype(int)

In [15]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
date_crawled          50000 non-null object
name                  50000 non-null object
price                 50000 non-null int64
ab_test               50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gearbox               47320 non-null object
power_ps              50000 non-null int64
model                 47242 non-null object
odometer_km           50000 non-null int64
registration_month    50000 non-null int64
fuel_type             45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
postal_code           50000 non-null int64
last_seen             50000 non-null object
dtypes: int64(6), object(11)
memory usage: 6.5+ MB


In [16]:
autos.max(axis=0, skipna=True)

date_crawled                             2016-04-07 14:36:56
name                  Älteres_Auto_zu_Verkaufen_aber_Laeuft.
price                                               99999999
ab_test                                                 test
registration_year                                       9999
power_ps                                               17700
odometer_km                                           150000
registration_month                                        12
brand                                                  volvo
ad_created                               2016-04-07 00:00:00
postal_code                                            99998
last_seen                                2016-04-07 14:58:50
dtype: object

In [17]:
autos.min(axis=0, skipna=True)

date_crawled                                        2016-03-05 14:06:30
name                  !!!!!!!!!!!!!!Renault_Clio_5_Tuerig_Servolenku...
price                                                                 0
ab_test                                                         control
registration_year                                                  1000
power_ps                                                              0
odometer_km                                                        5000
registration_month                                                    0
brand                                                        alfa_romeo
ad_created                                          2015-06-11 00:00:00
postal_code                                                        1067
last_seen                                           2016-03-05 14:45:46
dtype: object

In [45]:
print(autos['odometer_km'].unique().shape)
print(autos['odometer_km'].describe())
autos['odometer_km'].value_counts().sort_index(ascending=False).head(5)

(13,)
count     46681.000000
mean     125586.855466
std       39852.528628
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


150000    30085
125000     4857
100000     2058
90000      1673
80000      1375
Name: odometer_km, dtype: int64

In [47]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head(10)

(2323,)
count     46681.000000
mean       5977.716801
std        9177.909479
min           1.000000
25%        1250.000000
50%        3100.000000
75%        7500.000000
max      350000.000000
Name: price, dtype: float64


500     757
1500    696
2500    614
1200    606
1000    602
600     511
3500    480
800     470
2000    438
999     413
Name: price, dtype: int64

There are cars that have 0$ price and there are cars that have price greater than 400000. Let us clean up this data as it is not relevant for analysis here. 

In [42]:
autos['price'].value_counts().sort_index(ascending=False).head(10)

350000    1
345000    1
299000    1
295000    1
265000    1
259000    1
250000    1
220000    1
198000    1
197000    1
Name: price, dtype: int64

In [ ]:
autos = autos[autos['price'].between(0,350000)]

In [20]:
autos.shape

(49986, 17)

In [21]:
autos = autos[autos["price"].between(1,351000)]
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

# Explore Date fields

There are fields with date data in them. Lets explore them and clean them as necessary. If the distribution of listings are not in the range that we are looking for, delete those rows.

In [22]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [23]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In [24]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

In [25]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

# Incorrect registration data

It appears we have max value of 9999 in registration year, which is not possible. Let us look at this data and delete these records as they may skew the results.

In [26]:
autos['registration_year'].value_counts().sort_index(ascending=False)

9999       3
9000       1
8888       1
6200       1
5911       1
5000       4
4800       1
4500       1
4100       1
2800       1
2019       2
2018     470
2017    1392
2016    1220
2015     392
2014     663
2013     803
2012    1310
2011    1623
2010    1589
2009    2085
2008    2215
2007    2277
2006    2670
2005    2936
2004    2703
2003    2699
2002    2486
2001    2636
2000    3156
        ... 
1964      12
1963       8
1962       4
1961       6
1960      23
1959       6
1958       4
1957       2
1956       4
1955       2
1954       2
1953       1
1952       1
1951       2
1950       3
1948       1
1943       1
1941       2
1939       1
1938       1
1937       4
1934       2
1931       1
1929       1
1927       1
1910       5
1800       2
1111       1
1001       1
1000       1
Name: registration_year, Length: 95, dtype: int64

There are all sorts of years in the dataset, Let us select the years between 1900 and 2016. 

In [27]:
autos = autos[autos['registration_year'].between(1900,2016)]

In [28]:
autos['registration_year'].value_counts(normalize=True, dropna=False).sort_index()

1910    0.000107
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000043
1937    0.000086
1938    0.000021
1939    0.000021
1941    0.000043
1943    0.000021
1948    0.000021
1950    0.000064
1951    0.000043
1952    0.000021
1953    0.000021
1954    0.000043
1955    0.000043
1956    0.000086
1957    0.000043
1958    0.000086
1959    0.000129
1960    0.000493
1961    0.000129
1962    0.000086
1963    0.000171
1964    0.000257
1965    0.000364
1966    0.000471
1967    0.000557
1968    0.000557
          ...   
1987    0.001542
1988    0.002892
1989    0.003727
1990    0.007433
1991    0.007262
1992    0.007926
1993    0.009104
1994    0.013474
1995    0.026285
1996    0.029412
1997    0.041794
1998    0.050620
1999    0.062060
2000    0.067608
2001    0.056468
2002    0.053255
2003    0.057818
2004    0.057904
2005    0.062895
2006    0.057197
2007    0.048778
2008    0.047450
2009    0.044665
2010    0.034040
2011    0.034768
2012    0.028063
2013    0.017202
2014    0.0142

# Exploring price by Brand

Let's look at how brand value impacts the price and then we will look at if brand, mileage has any impact on price of the car.

In [29]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

German manufacturers account for more than 50% of the cars with volkswagen taking highest share of 21%. There are los of other brands at  less than 5% and are not important for our analysis. Let us limit our analysis to brands that have greater than 5%

In [30]:
brand_counts = autos['brand'].value_counts(normalize=True)

In [31]:
common_brands = brand_counts[brand_counts>0.05].index

In [32]:
common_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')

Volkswagen, bmw, opel, benz, audi and ford are the most popular brands. Let us look at the average prices for these brands. 

In [33]:
brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos['brand'] == brand]
    mean_price = brand_only['price'].mean()
    brand_mean_prices[brand] = int(mean_price)

brand_mean_prices

{'audi': 9336,
 'bmw': 8332,
 'ford': 3749,
 'mercedes_benz': 8628,
 'opel': 2975,
 'volkswagen': 5402}

Audi, BMW and Benz are the most expensive of the common brands whereas Ford and Opel are less expensive. Volkswagen is in the middle, which is why it is most popular as it offers both expesnive and less expensive auto mobils.

# Exploring mileage by brand

In [34]:
bmp_series = pd.Series(brand_mean_prices)

In [35]:
bmp_series

audi             9336
bmw              8332
ford             3749
mercedes_benz    8628
opel             2975
volkswagen       5402
dtype: int64

In [36]:
pd.DataFrame(bmp_series,columns=['mean_mileage'])

,mean_mileage
audi,9336
bmw,8332
ford,3749
mercedes_benz,8628
opel,2975
volkswagen,5402


In [37]:
brand_mean_mileage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [38]:
brand_info = pd.DataFrame(mean_mileage,columns=['mean_mileage'])
brand_info

,mean_mileage
bmw,132572
mercedes_benz,130788
opel,129310
audi,129157
volkswagen,128707
ford,124266


In [39]:
brand_info["mean_price"] = mean_prices
brand_info

,mean_mileage,mean_price
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
ford,124266,3749


Average mileage do not vary much compared to prices. There is a very slight trend that shows expensive cars having higher mileage whereas inexpensive ones have less mileage. 